## Description
_______

This script calculates statistics for the simulated vs. observed streamflow from the ESP outputs. 
Statistics currently being calculated are bias for each year included in the ESP analysis, correlation coefficient,
RMSE and NSE (Huang et al. 2017). These statistics are being calculated with the mean of the ensemble. 
The variables used for the calculations are described in the "Other Stitistics" cell so that additional stats can easily be added. Stats are calculated between start_date and end_date

### Import Libraries

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from scipy.integrate import simps

### Inputs

In [ ]:
# Define inputs for the plot
directory_path= '../58213_esp_results/' # directory containing ESP outputs
start_date= '04-01'       # start date for esp analysis in %Y-%m-%d
end_date= '04-30'         # end date for esp analysis in %Y-%m-%d
output_directory= '../'   # location for the outputs

In [ ]:
# adding computed runoff for given year (optional, replace with False if not using)
computed_path= None #'../0058213.txt'

### Generate Plot

In [ ]:
# Extract month and day from the dates
start_month, start_day = start_date.split('-')
end_month, end_day = end_date.split('-')

# Convert dates to integers
start_month, start_day = int(start_month), int(start_day)
end_month, end_day = int(end_month), int(end_day)

In [ ]:
# Initialize an empty list to store the total simulated runoff for each ensemble member
all_sum_cout_series = []

# Initialize an empty list to store the total observed runoff for each ensemble member
all_sum_rout_series = []

In [ ]:
if computed_path is not None:
    # Read the DataFrame from the computed_path
    sim = pd.read_csv(computed_path, sep='\t', index_col=0)

    sim = sim.drop('UNITS', axis=0)

    # Convert index to datetime format
    sim.index = pd.to_datetime(sim.index, errors='coerce')
    
    sum_model_sim= []
    
        # Convert 'cout' column to numeric if needed
    sim['cout'] = pd.to_numeric(sim['cout'], errors='coerce')


In [ ]:
# Iterate through each .nc file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.nc'):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Open the esp output file
        esp = xr.open_dataset(file_path)
        
        # Convert all data variable values to float
        esp = esp.astype(float)
        
        # Convert 'DATE' coordinate to datetime format
        esp['DATE'] = pd.to_datetime(esp['DATE'])
        
        # Extract the year from the last DATE
        last_date_year = pd.to_datetime(esp['DATE'][-1].values).year
        
        # Create start_date and end_date for the analysis period
        start_date = pd.Timestamp(year=last_date_year, month=start_month, day=start_day)
        end_date = pd.Timestamp(year=last_date_year, month=end_month, day=end_day)
        
        # Select data between start_date and end_date 
        ds_selected = esp.sel(DATE=slice(start_date, end_date))
        
        # Sum 'cout' variable for each ensemble member
        sum_cout = ds_selected['cout'].sum(dim='DATE')
        sum_rout= ds_selected['rout'].sum(dim='DATE')
        
        # Convert sum_cout to pandas Series
        sum_cout_series = sum_cout.to_series()
        sum_rout_series = sum_rout.to_series()
        
        # Ensure the simulated index is a DatetimeIndex and add year of analysis to series
        sum_cout_series.index = pd.to_datetime(sum_cout_series.index)
        sum_cout_series.index = sum_cout_series.index.map(lambda x: x.replace(year=last_date_year))
        
        # Ensure the observed index is a DatetimeIndex and add year of analysis to series
        sum_rout_series.index = pd.to_datetime(sum_rout_series.index)
        sum_rout_series.index = sum_rout_series.index.map(lambda x: x.replace(year=last_date_year))
        
        if computed_path is not None:
            # Trim the DataFrame based on the date range
            model_sim = sim.loc[start_date:end_date]
            
            # Sum the 'cout' column
            sum_sim = model_sim['cout'].sum()
            
            sim_series= sum_rout_series.copy()
            
            sim_series[:]= sum_sim
            
            sim_series.name = 'sim'
            
            sum_model_sim.append(sim_series)
        
        # Append the simulated and observed series to the list
        all_sum_cout_series.append(sum_cout_series)
        all_sum_rout_series.append(sum_rout_series)


In [ ]:
# Concatenate all sum_cout_series into a single series
sum_cout_series_combined = pd.concat(all_sum_cout_series)
sum_rout_series_combined = pd.concat(all_sum_rout_series)

In [ ]:
# Extract unique years from the computed runoff
unique_years = sum_cout_series_combined.index.year.unique()

In [ ]:
# Extract simulated data for each unique year 
yearly_cout = [sum_cout_series_combined[sum_cout_series_combined.index.year == year].values 
        for year in sum_cout_series_combined.index.year.unique()]

In [ ]:
yearly_rout= [sum_rout_series_combined[sum_rout_series_combined.index.year == year].values 
        for year in sum_rout_series_combined.index.year.unique()]

In [ ]:
# Calculate the mean for each year
mean_yearly_cout = [np.mean(year_data) for year_data in yearly_cout]
rout_value = [np.mean(year_data) for year_data in yearly_rout]

#### Calculate Statistics

In [ ]:
#def crps(observed, forecast):
#    n = len(forecast)
#    crps_values = []
#    
#    for i in range(n):
#        # Sort the forecast values
#        fc_sorted = np.sort(forecast[i])
#        
#        # Calculate the cumulative sum
#        fc_cdf = np.cumsum(fc_sorted) / np.sum(fc_sorted)
#        
#        # Calculate the CRPS using Simpson's rule
#        crps_value = simps((fc_cdf - (observed[i] / np.sum(fc_sorted)))**2, fc_sorted)
#        crps_values.append(crps_value)
#        
#        
#    return crps_values

In [ ]:
# Calculate CRPS
# crps_values = crps(rout_value, yearly_cout)

In [ ]:
# Create results dataframe
results = pd.DataFrame({
    'ESP Mean Total Flow (cms)': mean_yearly_cout,
    'Observed Total Flow (cms)': rout_value
}, index=unique_years)

In [ ]:
# Calculate bias 
results['Observed Bias'] = results['ESP Mean Total Flow (cms)'] - results['Observed Total Flow (cms)']

In [ ]:
# calculate mean bias across years
mean_bias = results['Observed Bias'].mean()

In [ ]:
# Calculate correlation coefficient for each row
correlation_coefficient = np.corrcoef(results['ESP Mean Total Flow (cms)'], results['Observed Total Flow (cms)'])[0, 1]

In [ ]:
# Calculate RMSE for each row
rmse = np.sqrt(np.mean((results['Observed Total Flow (cms)'] - results['ESP Mean Total Flow (cms)'])**2))

In [ ]:
# Calculate NSE
observed_mean = np.mean(results['Observed Total Flow (cms)'])
nse = 1 - (np.sum((results['Observed Total Flow (cms)'] - results['ESP Mean Total Flow (cms)'])**2) / 
           np.sum((results['Observed Total Flow (cms)'] - observed_mean)**2))

In [ ]:
# Create DataFrame
metrics = pd.DataFrame({
    'Observed Values': [correlation_coefficient, rmse, nse, mean_bias]
}, index=['Correlation Coefficient', 'RMSE', 'Nash Sutcliffe Efficiency', 'Mean Bias'])

In [ ]:
if computed_path is not None:
    # concat simulated values
    sum_model_sim_combined= pd.concat(sum_model_sim)
    
    # extract single value for each year
    yearly_computed= [sum_model_sim_combined[sum_model_sim_combined.index.year == year].values 
        for year in sum_model_sim_combined.index.year.unique()]
    
    computed_value = [np.mean(year_data) for year_data in yearly_computed]

    # Add 'Computed Total Flow (cms)' column to 'results'
    results['Simulated Total Flow (cms)'] = computed_value

    # Move 'Computed Total Flow (cms)' column to the third position
    cols = results.columns.tolist()
    cols.insert(2, cols.pop(cols.index('Simulated Total Flow (cms)')))
    results = results[cols]
    
    results['Simulated Bias'] = results['ESP Mean Total Flow (cms)'] - results['Simulated Total Flow (cms)']
    
    # calculate mean bias across years
    sim_mean_bias = results['Simulated Bias'].mean()
    
    # Calculate 'Percent Difference'
    results['Percent Difference'] = ((results['Simulated Bias'] - results['Observed Total Flow (cms)']) / 
                                 results['Observed Total Flow (cms)']) * 100
    
    # Calculate correlation coefficient for each row
    sim_correlation_coefficient = np.corrcoef(results['ESP Mean Total Flow (cms)'], results['Simulated Total Flow (cms)'])[0, 1]
    
    # Calculate RMSE for each row
    sim_rmse = np.sqrt(np.mean((results['Simulated Total Flow (cms)'] - results['ESP Mean Total Flow (cms)'])**2))
    
    # Calculate NSE
    sim_observed_mean = np.mean(results['Observed Total Flow (cms)'])
    sim_nse = 1 - (np.sum((results['Observed Total Flow (cms)'] - results['ESP Mean Total Flow (cms)'])**2) / 
               np.sum((results['Observed Total Flow (cms)'] - sim_observed_mean)**2))
    
    # Create DataFrame
    metrics = pd.DataFrame({
        'Observed Statistics': [correlation_coefficient, rmse, nse, mean_bias],
        'Simulated Statistics': [sim_correlation_coefficient, sim_rmse, sim_nse, sim_mean_bias]
     }, index=['Correlation Coefficient', 'RMSE', 'Nash Sutcliffe Efficiency', 'Mean Bias'])
    

### Other Statistics
_______
Compute any requred statistic by using:  
rout_value= Observed streamflow for each year  
mean_yearly_cout= Mean simulated streamflow for the ensemble   
yearly_cout= Ensemble of esp streamflows for each year  

#### Outputs

In [ ]:
# define outputs
bias_filename = 'esp_bias.csv'
bias_output_path = output_directory + bias_filename

statistics_filename= 'esp_stats.csv'
stats_output_path = output_directory + statistics_filename

In [ ]:
# Convert all values in DataFrames to floats
results = results.astype(float)
metrics = metrics.astype(float)

# Round all values to two decimal places
results = results.round(2)
metrics = metrics.round(2)

In [ ]:
# Save bias to CSV
results.to_csv(bias_output_path)

# save stats to CSV
metrics.to_csv(stats_output_path)